In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
text = ["London Paris London","Paris Paris London"]

In [5]:
cv = CountVectorizer()
#tells us that how many times a work appear in a sentence

In [8]:
count_matrix = cv.fit_transform(text)
count_matrix.toarray()

array([[2, 1],
       [1, 2]], dtype=int64)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
similarity_scores = cosine_similarity(count_matrix)
similarity_scores

array([[1. , 0.8],
       [0.8, 1. ]])

# Building the movie recommendation engine

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
df = pd.read_csv("movie_dataset.csv")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [25]:
features = ['keywords','cast','genres','director']

In [26]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

In [27]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string

df["combined_features"] = df.apply(combine_features,axis=1) 
# Applying combined_features() method over each rows of dataframe and 
# storing the combined string in "combined_features" column

In [32]:
df['combined_features'].head()[0]

'culture clash future space war space colony society Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Action Adventure Fantasy Science Fiction James Cameron'

In [33]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])

In [37]:
cosine_sim = cosine_similarity(count_matrix)

In [35]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [38]:
movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) 
#accessing the row corresponding to given movie to find all the similarity
# scores for that movie and then enumerating over it

In [39]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

In [41]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 5 similar movies to Avatar are:

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1
Star Trek Into Darkness
Star Trek Beyond
Alien
Lockout
Jason X
The Helix... Loaded
Moonraker
Planet of the Apes
Galaxy Quest
Gravity
Alien³
Jupiter Ascending
The Wolverine
Silent Running
Zathura: A Space Adventure
Trekkies
Cargo
Wing Commander
Star Trek
Lost in Space
Babylon A.D.
The Fifth Element
Oblivion
Titan A.E.
AVP: Alien vs. Predator
The Empire Strikes Back
Dragonball Evolution
Superman Returns
Divergent
John Carter
The Black Hole
The Ice Pirates
Memoirs of an Invisible Man
Starship Troopers
The Astronaut's Wife
Machete Kills
Soldier
The Abyss
Damnation Alley
Men in Black
Space Cowboys
Space Dogs
The Time Machine
Sheena
Captain America: Civil War
Star Trek: Insurrection
Oz: The Great and Powerful
The One
X-Men: Days of Future Past
Men in Black II
Return of the Jedi
Treasure Planet
Resident Evil
Star Trek III: The Search for Spock
Capricorn One
Elysium
Event Horizon
V for Vendetta


Womb
Space Battleship Yamato
MacGruber
Swelter
A Dog Of Flanders
Hard to Be a God
The Beastmaster
Decoys
Survival of the Dead
Creature
Silver Medalist
Containment
Airborne
A LEGO Brickumentary
Rotor DR1
Tadpole
Legend of the Guardians: The Owls of Ga'Hoole
Cloudy with a Chance of Meatballs 2
Tears of the Sun
The Specialist
The Crow
Safe
George and the Dragon
Baby Mama
Machine Gun Preacher
Return to Oz
The End of the Affair
Red Lights
Madison
Ong Bak 2
The Crocodile Hunter: Collision Course
Legend of a Rabbit
The Last Dragon
Atlas Shrugged Part II
The Other Side of Heaven
Falcon Rising
The Barbarians
A Guy Named Joe
Eddie: The Sleepwalking Cannibal
Def-Con 4
Bronson
The Hobbit: The Desolation of Smaug
Son of the Mask
Six Days Seven Nights
Hollywood Homicide
Sinbad: Legend of the Seven Seas
Muppets Most Wanted
Heartbreakers
The Man in the Iron Mask
Thunder and the House of Magic
The Quick and the Dead
Get on Up
Dangerous Liaisons
A Guy Thing
The Children of Huang Shi
Arn: The Knight Temp

The Pursuit of D.B. Cooper
Darling Companion
On the Line
Kites
The Wicked Lady
The Land Girls
Code of Honor
Pound of Flesh
The Hit List
Sunshine State
Exiled
3 Strikes
The Last Time I Committed Suicide
Compadres
Chicago Overcoat
Close Range
Do You Believe?
Dinner Rush
Dysfunctional Friends
Grace Unplugged
Wind Walkers
The Hebrew Hammer
Home Run
L.I.E.
Show Me
Compliance
Archaeology of a Woman
Unstoppable
Bullet to the Head
Hereafter
Inchon
Big Trouble
Ip Man 3
United Passions
Alatriste
Unaccompanied Minors
King's Ransom
Out of Inferno
The Last Station
Dickie Roberts: Former Child Star
Whatever It Takes
The Last Godfather
The Secret Life of Bees
Malone
Bucky Larson: Born to Be a Star
The Contender
Beneath Hill 60
Brooklyn Rules
Black November
Devil's Due
In Too Deep
Gerry
Shade
Nighthawks
Fascination
State Fair
Adore
Next Day Air
Go for It!
Yes
Out of the Dark
Incident at Loch Ness
House at the End of the Drive
The 41–Year–Old Virgin Who Knocked Up Sarah Marshall and Felt Superbad About

Final Destination
Hit & Run
My Fellow Americans
Beverly Hills Cop II
Beverly Hills Chihuahua
Are We There Yet?
Boogeyman
Monte Carlo
The Kite Runner
Armored
Crank: High Voltage
Birth
I Know What You Did Last Summer
Varsity Blues
American Beauty
Keanu
Buffalo Soldiers
Born on the Fourth of July
Exorcist II: The Heretic
Catch a Fire
The Lucky Ones
The Prisoner of Zenda
The Land Before Time
Aquamarine
Gremlins
Slackers
The Mighty Ducks
Gun Shy
Gunless
Set It Off
Silmido
Drive Me Crazy
Persepolis
Veer-Zaara
Dazed and Confused
Choke
Willy Wonka & the Chocolate Factory
Donovan's Reef
Waltz with Bashir
Gangster's Paradise: Jerusalema
The Dog Lover
Down Terrace
Mr. & Mrs. Smith
The Other Guys
The Lion King
Hostage
Dr. Dolittle 2
Body of Lies
The Kid
The Bourne Identity
The Family Man
Jack Ryan: Shadow Recruit
Shooter
Mortdecai
The Pacifier
Elizabethtown
22 Jump Street
Run All Night
My Best Friend's Wedding
Across the Universe
Foodfight!
One for the Money
Why I Did (Not) Eat My Father
Fifty Sha

Against the Ropes
Grudge Match
Sweet Home Alabama
The Ugly Truth
Water for Elephants
The Hurricane
Enough
Angel Eyes
Joe Somebody
The Ninth Gate
Extreme Measures
Precious
The Master
The Express
Creed
The Town
Burn After Reading
Magnolia
How to Be Single
Dolphin Tale
John Q
We're the Millers
Dolphin Tale 2
Reindeer Games
The Rite
Isn't She Great
Head of State
The Hangover
The Blind Side
Horrible Bosses
The Devil Wears Prada
Identity Thief
Cape Fear
21
Trainwreck
In & Out
A Nightmare on Elm Street
Secretariat
Radio
Friends with Benefits
Neighbors 2: Sorority Rising
Saving Mr. Banks
Malcolm X
This Is 40
Old Dogs
License to Wed
The Benchwarmers
Must Love Dogs
Poltergeist
The Ladykillers
The Back-Up Plan
Something Borrowed
The Bad News Bears
The Nativity Story
House of Wax
Closer
J. Edgar
Untraceable
Blast from the Past
Jersey Girl
Midnight in the Garden of Good and Evil
Hoffa
Concussion
Wonder Boys
Broken City
Cursed
Furry Vengeance
Walk Hard: The Dewey Cox Story
The Shipping News
The Youn

Black Snake Moan
A Mighty Heart
Boat Trip
The Importance of Being Earnest
Hoot
In Bruges
The Rocker
Post Grad
Promised Land
Whatever Works
The In Crowd
Jakob the Liar
Idle Hands
Mulholland Drive
You Will Meet a Tall Dark Stranger
The Company
Transsiberian
Crazy in Alabama
Funny Games
Felicia's Journey
Things to Do in Denver When You're Dead
The Return
The Midnight Meat Train
Chiamatemi Francesco - Il Papa della gente
Pandaemonium
Magic Mike XXL
Romeo + Juliet
Freedom
Unforgiven
Manderlay
Slumdog Millionaire
Fatal Attraction
Pretty Woman
Broken Horses
Cool Runnings
My Bloody Valentine
Stomp the Yard
Urban Legend
Good Deeds
Superstar
The Iron Lady
Poetic Justice
Vampire in Brooklyn
My Boss's Daughter
Our Family Wedding
Dead Man on Campus
Thinner
New York, New York
Crooklyn
I Think I Love My Wife
Bobby
Head Over Heels
Fun Size
The Diving Bell and the Butterfly
Little Children
A Walk on the Moon
Easy Virtue
Mr. Turner
Wild Grass
Amen.
Margaret
Stan Helsing
Flipped
Clueless
Far from Heaven


Lovely, Still
Desert Blue
The Visit
Sleep Tight
Farce of the Penguins
Flying By
Rudderless
Henry & Me
We Have Your Husband
Dying of the Light
Should've Been Romeo
Running Forever
I Served the King of England
Soul Kitchen
Sling Blade
The Awakening
Hostel
A Cock and Bull Story
Take Shelter
Lady in White
Let's Kill Ward's Wife
The Texas Chainsaw Massacre 2
Only God Forgives
Camping Sauvage
Without Men
Dear Frankie
The Names of Love
Treading Water
Out of the Blue
Police Academy
Fast Times at Ridgemont High
25th Hour
Bound
Requiem for a Dream
Tango
Moms' Night Out
Character
Spun
Life During Wartime
Sympathy for Lady Vengeance
Mozart's Sister
Mean Machine
Lilya 4-ever
After.Life
Fugly
One Flew Over the Cuckoo's Nest
Silent Movie
Anne of Green Gables
Easy Money
Whale Rider
Paa
High School Musical
Love and Death on Long Island
Porky's
Night of the Living Dead
Lost in Translation
Annie Hall
Monster's Ball
Maggie
Leaving Las Vegas
Hansel and Gretel Get Baked
The Front Page
The Boy Next Door
Trap

Horse Camp
Give Me Shelter
Little Big Top
Along the Roadside
Burn
Urbania
The Stewardesses
Swingers
She Done Him Wrong
Short Cut to Nirvana: Kumbh Mela
The Grace Card
Middle of Nowhere
Call + Response
Malevolence
Reality Show
Super Hybrid
Baghead
American Beast
The Case of the Grinning Cat
Ordet
Good Dick
The Trials Of Darryl Hunt
Samantha: An American Girl Holiday
Theresa Is a Mother
Children of Heaven
Weekend
She's Gotta Have It
Butterfly Girl
The World Is Mine
Perfect Cowboy
Once
The Woman Chaser
The Horse Boy
When the Lights Went Out
A Charlie Brown Christmas
Antarctic Edge: 70° South
Aroused
Top Spin
Roger & Me
An American in Hollywood
The Blood of My Brother: A Story of Death in Iraq
Your Sister's Sister
A Dog's Breakfast
The Married Woman
The Birth of a Nation
The Work and The Story
The Gallows
Hollywood Shuffle
The Mighty
Cheap Thrills
Indie Game: The Movie
Straightheads
Open Secret
The Night Visitor
The Past Is a Grotesque Animal
Peace, Propaganda & the Promised Land
Pi
I Love